# Data Merging & Preparation for ML Model

## 🎯 Objective

This notebook merges all 4 datasets (fire, weather, drought, population) into a **unified dataset** ready for exploratory data analysis and machine learning modeling.

## 📊 What This Notebook Does

1. **Loads processed fire data** from CSV (complete unit-month grid)
2. **Loads weather data** from NOAA Climate at a Glance (2000-2025)
3. **Loads drought data** from US Drought Monitor (2000-2025)
4. **Loads population data** from CA Department of Finance (2000-2025)
5. **Merges everything** into single dataframe for ML modeling

## 🔥 Key Change: Using Processed Fire CSV

We now use the processed fire CSV file instead of raw geodatabase data for faster and cleaner processing.


In [29]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

print('✅ All libraries imported successfully!')
print('🔥 Using processed fire CSV file (no more raw data!)')


✅ All libraries imported successfully!
🔥 Using processed fire CSV file (no more raw data!)


In [30]:
# Step 1: Load Fire Data from Processed CSV
print('='*60)
print('STEP 1: LOADING FIRE DATA FROM PROCESSED CSV')
print('='*60)

# Load processed fire data from CSV (with complete unit-month grid)
fire_csv_path = Path('../data/processed/fire_data_complete_unit_month.csv')
fires_df = pd.read_csv(fire_csv_path)

print(f'✅ Loaded {len(fires_df):,} fire records from processed CSV')
print(f'   Shape: {fires_df.shape}')
print(f'   Available columns: {list(fires_df.columns)}')

# Show dataset header and sample data
print(f'\nDataset header (first 5 rows):')
print(fires_df.head(5))

print(f'\nDataset info:')
print(f'   Shape: {fires_df.shape}')
print(f'   Columns: {list(fires_df.columns)}')

print(f'\nSample fire data (key columns):')
print(fires_df[['Unit_ID', 'County', 'Year', 'Month', 'Fire_Occurred', 'Fire_Count']].head(5))

# Check data statistics
print(f'\n✅ Fire data statistics:')
print(f'   Date range: {fires_df["Year"].min()}-{fires_df["Year"].max()}')
print(f'   Unique units: {fires_df["Unit_ID"].nunique()}')
print(f'   Unique counties: {fires_df["County"].nunique()}')
print(f'   Fire occurrence rate: {(fires_df["Fire_Occurred"] == 1).mean()*100:.2f}%')

# Check county mapping
mapped_counties = fires_df['County'].notna().sum()
total_records = len(fires_df)
print(f'   County mapping: {mapped_counties:,}/{total_records:,} records ({mapped_counties/total_records*100:.1f}%)')

print(f'\n🎯 Fire data is ready for merging with other datasets!')


STEP 1: LOADING FIRE DATA FROM PROCESSED CSV
✅ Loaded 34,008 fire records from processed CSV
   Shape: (34008, 9)
   Available columns: ['Unit_ID', 'Year', 'Month', 'Fire_Count', 'Total_Acres', 'Avg_Acres', 'Max_Acres', 'Fire_Occurred', 'County']

Dataset header (first 5 rows):
  Unit_ID  Year  Month  Fire_Count  Total_Acres  Avg_Acres  Max_Acres  \
0     ADR  2000      1         0.0          0.0        0.0        0.0   
1     ADR  2000      2         0.0          0.0        0.0        0.0   
2     ADR  2000      3         0.0          0.0        0.0        0.0   
3     ADR  2000      4         0.0          0.0        0.0        0.0   
4     ADR  2000      5         0.0          0.0        0.0        0.0   

   Fire_Occurred  County  
0            0.0  ALPINE  
1            0.0  ALPINE  
2            0.0  ALPINE  
3            0.0  ALPINE  
4            0.0  ALPINE  

Dataset info:
   Shape: (34008, 9)
   Columns: ['Unit_ID', 'Year', 'Month', 'Fire_Count', 'Total_Acres', 'Avg_Acres', '

In [31]:
fires_df.head(5)

,Unit_ID,Year,Month,Fire_Count,Total_Acres,Avg_Acres,Max_Acres,Fire_Occurred,County
0,ADR,2000,1,0.0,0.0,0.0,0.0,0.0,ALPINE
1,ADR,2000,2,0.0,0.0,0.0,0.0,0.0,ALPINE
2,ADR,2000,3,0.0,0.0,0.0,0.0,0.0,ALPINE
3,ADR,2000,4,0.0,0.0,0.0,0.0,0.0,ALPINE
4,ADR,2000,5,0.0,0.0,0.0,0.0,0.0,ALPINE


In [36]:
# 1. Get unique values.
unique_values = fires_df['County'].unique()

# 2. Convert all elements to strings (str)
#    This handles the mix of floats and strings.
unique_strings = [str(x) for x in unique_values]

# 3. Sort the list of strings
sorted_counties = sorted(unique_strings)

# Print the result
print(sorted_counties)

['ALAMEDA', 'ALPINE', 'AMADOR', 'BUTTE', 'COLUSA', 'CONTRA COSTA', 'DEL NORTE', 'EL DORADO', 'FRESNO', 'GLENN', 'HUMBOLDT', 'IMPERIAL', 'INYO', 'KERN', 'LAKE', 'LOS ANGELES', 'MADERA', 'MARIN', 'MARIPOSA', 'MENDOCINO', 'NEVADA', 'PLACER', 'PLUMAS', 'RIVERSIDE', 'SACRAMENTO', 'SAN BERNARDINO', 'SAN DIEGO', 'SAN JOAQUIN', 'SAN LUIS OBISPO', 'SAN MATEO', 'SANTA CLARA', 'SANTA CRUZ', 'SHASTA', 'SISKIYOU', 'SOLANO', 'SONOMA', 'STANISLAUS', 'TEHAMA', 'TRINITY', 'TULARE', 'TUOLUMNE', 'VENTURA', 'YOLO', 'nan']


In [41]:
fires_df.County.nunique()

43

In [10]:
# Step 2: Load Weather Data
print('='*60)
print('STEP 2: LOADING WEATHER DATA')
print('='*60)

# Load weather data from CSV files
weather_dir = Path('../data/raw/weather')

# Load all 4 weather CSV files
avg_temp_file = weather_dir / 'California-Average-Monthly-Temperature-2000-2025.csv'
max_temp_file = weather_dir / 'California-Max-Monthly-Temperature-2000-2025.csv'
min_temp_file = weather_dir / 'California-Min-Monthly-Temperature-2000-2025.csv'
precip_file = weather_dir / 'California-Monthly-Precipitation-2000-2025.csv'

print('Loading California climate data (2000-2025)...')

# Load each dataset
avg_temp_df = pd.read_csv(avg_temp_file, skiprows=4)  # Skip comment lines
max_temp_df = pd.read_csv(max_temp_file, skiprows=4)
min_temp_df = pd.read_csv(min_temp_file, skiprows=4)
precip_df = pd.read_csv(precip_file, skiprows=4)

print(f"✅ Loaded all 4 weather datasets")

# Check column names first
print(f'Avg temp columns: {list(avg_temp_df.columns)}')
print(f'Max temp columns: {list(max_temp_df.columns)}')
print(f'Min temp columns: {list(min_temp_df.columns)}')
print(f'Precip columns: {list(precip_df.columns)}')

# Combine into single dataframe
climate_df = pd.concat([
    avg_temp_df.rename(columns={'Value': 'Avg_Temp_F'}),
    max_temp_df.rename(columns={'Value': 'Max_Temp_F'}),
    min_temp_df.rename(columns={'Value': 'Min_Temp_F'}),
    precip_df.rename(columns={'Value': 'Precip_Inches'})
], axis=1)



STEP 2: LOADING WEATHER DATA
Loading California climate data (2000-2025)...
✅ Loaded all 4 weather datasets
Avg temp columns: ['200002', '47.2']
Max temp columns: ['200002', '56']
Min temp columns: ['200002', '38.4']
Precip columns: ['200002', '6.93']


In [11]:
avg_temp_df.head(5)

,200002,47.2
0,200003,50.2
1,200004,57.8
2,200005,63.9
3,200006,72.2
4,200007,73.5


In [5]:
# Select only the columns we want
climate_df = climate_df[['Date', 'Avg_Temp_F', 'Max_Temp_F', 'Min_Temp_F', 'Precip_Inches']]

# Parse dates and add Year/Month columns
climate_df['Date'] = pd.to_datetime(climate_df['Date'])
climate_df['Year'] = climate_df['Date'].dt.year
climate_df['Month'] = climate_df['Date'].dt.month

print(f'✅ Weather data loaded: {len(climate_df)} records')
print(f'   Date range: {climate_df["Date"].min()} to {climate_df["Date"].max()}')

# Show dataset header
print(f'\nWeather dataset header (first 5 rows):')
print(climate_df.head(5))

print(f'\nDataset info:')
print(f'   Shape: {climate_df.shape}')
print(f'   Columns: {list(climate_df.columns)}')

print(f'\nSample weather data:')
print(climate_df[['Date', 'Year', 'Month', 'Avg_Temp_F', 'Max_Temp_F', 'Min_Temp_F', 'Precip_Inches']].head(5))

print(f'\n🎯 Weather data is ready for merging!')


KeyError: "None of [Index(['Date', 'Avg_Temp_F', 'Max_Temp_F', 'Min_Temp_F', 'Precip_Inches'], dtype='object')] are in the [columns]"